### Problem 9 : Quantum Support Vector Machine (QSVM)
- Support Vector Machines (SVMs) are powerful for classification tasks in classical ML. 
- Quantum Support Vector Machines (QSVMs) leverage quantum kernels to encode data into high-dimensional Hilbert space and compute similarities more efficiently.

#### To Do :

- **Problem Setup:** Classify simple 2D data points using a quantum kernel. Example: Two classes (labeled 0 and 1) with separable points.
- **Quantum Feature Map:** Encode classical data into quantum states using parameterised circuits.
- **Quantum Kernel Computation:** Compute pairwise similarity between data points using quantum circuits for training and testing points.
- **Train QSVM:** Use the quantum kernel matrix to train a classical SVM.


**MUST Read for reference:**
 - https://medium.com/@devmallyakarar/quantum-support-vector-machines-qsvm-using-qiskit-eee347e81d83
 - https://github.com/PatrickHuembeli/QSVM-Introduction/blob/master/Quantum%20Support%20Vector%20Machines.ipynb

In [1]:
# !pip install qiskit qiskit-machine-learning scikit-learn numpy

In [2]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.primitives import StatevectorSampler

from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np

In [3]:
# Step 1: Generate a Simple Dataset
def create_dataset():
    """Generate a simple binary classification dataset."""
    X, y = make_classification(
        n_samples=20, 
        n_features=2, 
        n_classes=2, 
        n_informative=2, 
        n_redundant=0, 
        random_state=42
    )
    return train_test_split(X, y, test_size=0.3, random_state=42)

The **ZZFeatureMap** is a quantum circuit that helps encode classical data into quantum states. Let me explain it simply:

It gets its name "ZZ" from how it works:
1. First, it applies single-qubit rotations (Z gates) based on your input features
2. Then it applies two-qubit ZZ operations between pairs of qubits, which creates entanglement
3. This pattern can be repeated multiple times (that's what the "reps" parameter controls)

Think of it like this - if you have classical data points (like coordinates in 2D space), the ZZFeatureMap transforms them into quantum states in a way that:
- Similar classical points should produce similar quantum states
- Different classical points should produce different quantum states

It's particularly useful for quantum machine learning because it can capture non-linear relationships in your data through quantum entanglement, which is exactly what you want for tasks like classification.

When you set `feature_dimension=2` and `entanglement='linear'` like in the code below, it means:
- It will handle 2D input data (points with x and y coordinates)
- The entangling operations will be applied in a linear chain (first qubit entangled with second, second with third, etc.)


In [4]:
def create_quantum_kernel():
    """Create a quantum kernel using ZZFeatureMap."""
    feature_map = ZZFeatureMap(
        feature_dimension=2,
        reps=2,
        entanglement='linear'
    )
    
    # Return just the kernel object, not a tuple
    return FidelityQuantumKernel(
        feature_map=feature_map
    )

A fidelity based quantum kernel FidelityQuantumKernel is used here. This is a direct replacement of the already existing QuantumKernel. The difference is that the new class takes a fidelity instance to estimate overlaps and construct a kernel matrix.

**Fidelity** is a measure of how close the final quantum state of the real-life qubits is to the ideal case. If the fidelity of logic gates is too low, calculations will fail because errors will accumulate faster than they can be corrected.

In [5]:
quantum_kernel = create_quantum_kernel()
print(type(quantum_kernel))

<class 'qiskit_machine_learning.kernels.fidelity_quantum_kernel.FidelityQuantumKernel'>


In [6]:
def train_and_evaluate_qsvm(X_train, X_test, y_train, y_test, quantum_kernel):
    """Train and evaluate the quantum SVM model."""
    
    # Compute kernel matrices
    kernel_matrix_train = quantum_kernel.evaluate(X_train, X_train)
    kernel_matrix_test = quantum_kernel.evaluate(X_test, X_train)
    
    # Train SVM with quantum kernel
    svm = SVC(kernel='precomputed')
    svm.fit(kernel_matrix_train, y_train)
    
    # Make predictions
    y_pred = svm.predict(kernel_matrix_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return y_pred, accuracy

In [7]:
# Step 1: Generate dataset
X_train, X_test, y_train, y_test = create_dataset()

# Step 2: Create quantum kernel
quantum_kernel = create_quantum_kernel()

# Step 3: Train and evaluate model
y_pred, accuracy = train_and_evaluate_qsvm(
    X_train, X_test, y_train, y_test, quantum_kernel
)

# Display results
print("Test Accuracy:", accuracy)
print("Predicted Labels:", y_pred)
print("True Labels:", y_test)

Test Accuracy: 0.5
Predicted Labels: [1 1 1 0 0 1]
True Labels: [0 0 1 0 0 0]


#### Expected Output
- Accuracy: Percentage of correctly classified test points.
- Predicted Labels: Classification results from the QSVM.
- True Labels: Ground truth for the test data.